## 꿈꾸는아이(AI) 전력 수요량 예측 경진대회
## Fe lab. 팀


## 1. 라이브러리 및 데이터
* 사용 library 및 version:
pandas==1.1.4
scikit-learn==0.23.2
tensorflow==2.1.0
* seed=42
* data path: 'data/'
* parameter path: 'data/parameters/'
* test.csv만을 이용하여 학습 및 예측

In [6]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

data_path = 'data/'

# 시드 설정 및 데이터 load
tf.compat.v1.set_random_seed(42)
np.random.seed(42)
test = pd.read_csv(data_path+'test.csv')
test['Time'] = pd.to_datetime(test.Time)
test = test.set_index('Time')
submission = pd.read_csv(data_path+'submission.csv')
submission.index = submission['meter_id']
submission.drop(columns=['meter_id'], inplace=True)

# Load tuned parameters
dct_param_list = pd.read_csv(data_path+'parameters/dct_params.csv',dtype={'max_depth':int,'min_samples_leaf':int})
dct_param_list.set_index('Unnamed: 0', inplace=True)
dnn_param_list = pd.read_csv(data_path+'parameters/dnn_params.csv',dtype={'EPOCH':int,'h1':int,'h2':int,'h3':int,'h4':int})
dnn_param_list.set_index('Unnamed: 0', inplace=True)
extra_param_list = pd.read_csv(data_path+'parameters/extra_params.csv',dtype={'max_depth':int,'min_samples_leaf':int,'n_jobs':int})
extra_param_list.set_index('Unnamed: 0', inplace=True)
rf_param_list = pd.read_csv(data_path+'parameters/rf_params.csv',dtype={'max_depth':int,'min_samples_leaf':int,'n_estimators':int,'random_state':int,'n_jobs':int})
rf_param_list.set_index('Unnamed: 0', inplace=True)
svr_param_list = pd.read_csv(data_path+'parameters/svr_params.csv')
svr_param_list.set_index('Unnamed: 0', inplace=True)

## 2. 함수 구성
* __smape__: smape 계산 모듈 
* __tr_data_set__: 시간별 예측을 위한 데이터 정리
* __tr_day_set__: 일별, 월별 예측을 위한 데이터 정리
* __linear_prediction and AR_val__: 선형 예측 모듈
* __similar_approach_val__: similar day 기반 예측 모듈
* __dnn_gen_val__: Multi Layer Perceptron (MLP) 예측 모듈
* __lstm_gen_val__: Long Short Term Memory (LSTM) 예측 모듈
* __rf_gen_val__: Random Forest (RF) 예측 모듈
* __svr_gen_val__: Support Vector Regression (SVR) 예측 모듈
* __extra_gen_val__: Extra tree 예측 모듈
* __dct_gen_val__: Decision Tree 예측 모듈

In [7]:
def smape(true, pred):
    """
    [ Input ]
    true: real data
    pred: predicted data
    
    [ Output ]
    output: smape value
    """
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

def tr_data_set(Data, place_id, prev_type, Curr_type):
    """
    [ Input ]
    Data: test.csv 전력 데이터
    place_id: 집 index
    prev_type: 전날의 daytype
    Curr_type: 현재 daytype
    
    [ Output ]
    TrainAR: 예측 전날 데이터 (shpae: n_day, 24_point)
    TestAR: 예측 날 데이터 (shpae: n_day, 24_point)
    """
    TrainAR = []
    TestAR = []
    len_bad = 20  # 하루에 NaN이 len_bad 이상이면 데이터 삭제
    Power = Data[place_id].iloc  # 전력
    Date = Data[place_id].index  # 날짜
    prev_aloc = [0] * 24
    curr_aloc = [0] * 24

    for ii in range(24, len(Date)):
        if (Date[ii].hour == 0) & (ii > 48) & (np.sum(curr_aloc) != 24 * curr_aloc[1]) & (
                np.sum(prev_aloc) != 24 * prev_aloc[1]):
            prev_idx = 0
            curr_idx = 0

            for kk in range(0, 24):
                # bad data인지 확인
                if prev_aloc[kk] > -20:
                    prev_idx = prev_idx + 1
                else:  # Bad data를 기준으로 앞뒤로 len_bad point를 가져와서 복원
                    temp = np.zeros([1, 41])
                    for qq in range(0, 41):
                        temp[0, qq] = Power[(ii - 24) - (24 - kk) - 20 + qq]

                    temp_temp = pd.DataFrame(data=temp)
                    # spline으로 interpolation
                    temp = temp_temp.interpolate('spline', order=1)
                    temp = temp.values
                    prev_aloc[kk] = temp[0, 20]

            for kk in range(0, 24):
                # bad data인지 확인
                if curr_aloc[kk] > -20:
                    curr_idx = curr_idx + 1
                else:
                    # Bad data를 기준으로 앞뒤로 len_bad point를 가져와서 복원
                    temp = np.zeros([1, 41])
                    for qq in range(0, 41):
                        temp[0, qq] = Power[(ii) - (24 - kk) - 20 + qq]
                    temp_temp = pd.DataFrame(data=temp)
                    # spline으로 interpolation
                    temp = temp_temp.interpolate('spline', order=1)
                    temp = temp.values
                    curr_aloc[kk] = temp[0, 20]

            # len_bad 이상인지 확인
            if (prev_idx > len_bad) & (curr_idx > len_bad):
                TrainAR.append(prev_aloc)
                TestAR.append(curr_aloc)

        # 0시에 데이터 초기화
        if Date[ii].hour == 0:
            prev_aloc = [0] * 24
            curr_aloc = [0] * 24

        # 무슨 요일인지 확인
        prev_day = Date[ii - 24].weekday()
        curr_day = Date[ii].weekday()

        # 요일 데이터 타입 분류
        # working day (1) = day type<5(월~금)
        # non-working day (2) = day type>4(토~일)
        if ((prev_type == 1) & (prev_day < 5)) & ((Curr_type == 2) & (curr_day > 4)):
            prev_aloc[Date[ii - 24].hour] = Power[ii - 24]
            curr_aloc[Date[ii].hour] = Power[ii]

        if ((prev_type == 1) & (prev_day < 5)) & ((Curr_type == 1) & (curr_day < 5)):
            prev_aloc[Date[ii - 24].hour] = Power[ii - 24]
            curr_aloc[Date[ii].hour] = Power[ii]

        if ((prev_type == 2) & (prev_day > 4)) & ((Curr_type == 2) & (curr_day > 4)):
            prev_aloc[Date[ii - 24].hour] = Power[ii - 24]
            curr_aloc[Date[ii].hour] = Power[ii]

        if ((prev_type == 2) & (prev_day > 4)) & ((Curr_type == 1) & (curr_day < 5)):
            prev_aloc[Date[ii - 24].hour] = Power[ii - 24]
            curr_aloc[Date[ii].hour] = Power[ii]

    TrainAR = np.array(TrainAR)
    TestAR = np.array(TestAR)
    return TrainAR, TestAR

def tr_day_set(data, place_id):
    """
    [ Input ]
    data: test.csv 전력 데이터
    place_id: 집 index
    
    [ Output ]
    temp_day: power consumption of previous day
    """
    Power = data[place_id].values  # 전력
    Date = data[place_id].index  # 요일

    temp_day = np.zeros([6, 7])
    mon_idx = np.zeros([1, 7])

    for ii in range(0, len(Power) - 500):

        idx = len(Power) - ii - 1

        day_idx = Date[idx].weekday()  # 요일 idx
        time_idx = Date[idx].hour  # 시간 idx

        if mon_idx[0, day_idx] < 6:  # 6주 까지만 남김
            if np.isnan(Power[idx]):  # 이상치 족원
                res_data = np.zeros([1, 9])
                # 같은 요일, 시간의 전력 데이터의 평균을 취함
                res_data[0, 0] = Power[idx - 24 * 7 - 1]
                res_data[0, 1] = Power[idx - 24 * 7]
                res_data[0, 2] = Power[idx - 24 * 7 + 1]

                res_data[0, 3] = Power[idx - 48 * 7 - 1]
                res_data[0, 4] = Power[idx - 48 * 7]
                res_data[0, 5] = Power[idx - 48 * 7 + 1]

                res_data[0, 6] = Power[idx - 1]
                res_data[0, 7] = Power[idx - 3 * 24 * 7]
                res_data[0, 8] = Power[idx + 1]
                # 하루 (0~24시)의 전력데이터를 더함
                temp_day[int(round(mon_idx[0, day_idx])), day_idx] = temp_day[int(
                    round(mon_idx[0, day_idx])), day_idx] + np.nanmean(res_data)
            else:
                # 하루 (0~24시)의 전력데이터를 더함
                temp_day[int(round(mon_idx[0, day_idx])), day_idx] = temp_day[
                                                                         int(round(mon_idx[0, day_idx])), day_idx] + \
                                                                     Power[idx]
            if time_idx == 0:
                # week index update
                mon_idx[0, day_idx] = mon_idx[0, day_idx] + 1

    return temp_day

def linear_prediction(trainAR, testAR, flen, test_data):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    flen: filter length
    test_data: 예측하고자 하는 날의 전 날 데이터
    
    [ Output ]
    avr_smape: validation set에 대한 평균 smape
    fcst: 추정하고자하는 날에 대한 예측 프로파일
    pred: validation set에 대한 예측 결과
    """
    # 선형 예측
    len_tr = len(trainAR[0, :])
    day_t = len(trainAR)
    pred = np.empty((len(trainAR), len_tr))
    fcst = np.empty((len(trainAR), len_tr))

    for j in range(0, day_t):
        if day_t > 1:
            x_ar = np.delete(trainAR[:, len_tr - flen:len_tr], (j), axis=0)
            y = np.delete(testAR, (j), axis=0)
        else:
            x_ar = trainAR[:, len_tr - flen:len_tr]
            y = testAR

        pi_x_ar = np.linalg.pinv(x_ar)
        lpc_c = np.empty((len(x_ar), flen))

        lpc_c = np.matmul(pi_x_ar, y)

        test_e = trainAR[j, :]
        test_ex = test_e[len_tr - flen:len_tr]
        pred[j, :] = np.matmul(test_ex, lpc_c)

    x_ar = trainAR[:, len_tr - flen:len_tr]
    y = testAR
    pi_x_ar = np.linalg.pinv(x_ar)
    lpc_c = np.empty((len(x_ar), flen))

    lpc_c = np.matmul(pi_x_ar, y)

    Test_AR = testAR[0:len(testAR), :]

    smape_list = np.zeros((len(pred), 1))

    for i in range(0, len(pred)):
        smape_list[i] = smape(pred[i, :], Test_AR[i, :])
        avr_smape = np.mean(smape_list)

    test_e = test_data
    test_ex = test_e[len_tr - flen:len_tr]
    fcst = np.matmul(test_ex, lpc_c)

    return avr_smape, fcst, pred

def AR_val(trainAR, testAR, flen, subm_24hrs):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    flen: filter length
    subm_24hrs: 예측하고자 하는 날의 전 날 데이터
    
    [ Output ]
    fcst: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set에 대한 평균 smape
    pred[numTr:,:]: validation 예측 결과
    """
    len_tr = len(trainAR[0, :])
    day_t = len(trainAR)
    numTr = int(day_t * 0.8)
    pred = np.empty((len(trainAR), len_tr))
    fcst = np.empty((len(trainAR), len_tr))

    for j in range(0, day_t):
        if day_t > 1:
            x_ar = np.delete(trainAR[:, len_tr - flen:len_tr], (j), axis=0)
            y = np.delete(testAR, (j), axis=0)
        else:
            x_ar = trainAR[:, len_tr - flen:len_tr]
            y = testAR
        pi_x_ar = np.linalg.pinv(x_ar)
        lpc_c = np.empty((len(x_ar), flen))
        lpc_c = np.matmul(pi_x_ar, y)
        test_e = trainAR[j, :]
        test_ex = test_e[len_tr - flen:len_tr]
        pred[j, :] = np.matmul(test_ex, lpc_c)

    x_ar = trainAR[:, len_tr - flen:len_tr]
    y = testAR
    pi_x_ar = np.linalg.pinv(x_ar)
    lpc_c = np.empty((len(x_ar), flen))
    lpc_c = np.matmul(pi_x_ar, y)
    Test_AR = testAR[0:len(testAR), :]
    smape_list = np.zeros((len(pred), 1))

    for i in range(0, len(pred)):
        smape_list[i] = smape(pred[i, :], Test_AR[i, :])
        avr_smape = np.mean(smape_list)

    test_e = subm_24hrs
    test_ex = test_e[len_tr - flen:len_tr]
    fcst = np.matmul(test_ex, lpc_c)

    return fcst, avr_smape, pred[numTr:,:]

def similar_approach_val(trainAR, testAR, slen, sim_set):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    slen: 사용할 날의 길이
    sim_set: 예측 전 날의 데이터
    
    [ Output ]
    simil_temp: 추정하고자하는 날에 대한 예측 프로파일
    simil_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    simil_smape_list = np.zeros([1, len(testAR[:, 0])])
    numTr = int(trainAR.shape[0] * 0.8)
    
    y_pred = [] # validation 예측 결과 저장
    for col_ii in range(0, len(testAR[:, 0])):
        simil_mean = []
        simil_temp = np.zeros([1, 24])
        simil_idx = np.zeros([1, len(testAR[:, 0])])

        for sub_col in range(0, len(testAR[:, 0])):
            simil_idx[0, sub_col] = smape(trainAR[col_ii, :], trainAR[sub_col, :])

        testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
        simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)

        for search_len in range(0, slen):
            simil_mean.append(testAR_temp[np.argmin(simil_idx), :])
            testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
            simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)

        for row_ii in range(0, 24):
            simil_temp[0, row_ii] = np.median(testAR_temp[:, row_ii])

        simil_smape_list[0, col_ii] = smape(testAR[col_ii, :], simil_temp)
        simil_smape = np.mean(simil_smape_list)
        if col_ii >= numTr:
            y_pred.append(simil_temp)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred,(-1,24))

    simil_mean = []
    simil_temp = np.zeros([1, 24])
    simil_idx = np.zeros([1, len(testAR[:, 0])])
    testAR_temp = testAR

    for sub_col in range(0, len(testAR[:, 0])):
        simil_idx[0, sub_col] = smape(sim_set, trainAR[sub_col, :])

    for search_len in range(0, slen):
        simil_mean.append(testAR_temp[np.argmin(simil_idx), :])
        testAR_temp = np.delete(testAR, np.argmin(simil_idx), axis=0)
        simil_idx = np.delete(simil_idx, np.argmin(simil_idx), axis=1)

    for row_ii in range(0, 24):
        simil_temp[0, row_ii] = np.median(testAR_temp[:, row_ii])

    return simil_temp, simil_smape, y_pred

def dnn_gen_val(trainAR, testAR, x_24hrs, params=None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    # default parameter
    if params == None:
        params = {
            'EPOCH': 80,
            'h1': 128,
            'h2': 256,
            'h3': 128,
            'h4': 0,
            'lr': 0.001,
        }

    # load data
    numData = np.size(trainAR, 0)
    numTr = int(numData * 0.8)
    Xtr = trainAR[0:numTr, :]
    Ytr = testAR[0:numTr, :]
    Xte = trainAR[numTr:numData, :]
    Yte = testAR[numTr:numData, :]
    num_tr = np.size(trainAR, 1)
    num_te = np.size(testAR, 1)

    # Build model
    model = tf.keras.Sequential()
    model.add(layers.Dense(params['h1'], activation='relu', input_shape=(num_tr,)))
    model.add(layers.Dense(params['h2'], activation='relu'))
    model.add(layers.Dense(params['h3'], activation='relu'))
    if params['h4'] != 0:
        model.add(layers.Dense(params['h4'], activation='relu'))
    model.add(layers.Dense(num_te))
    optimizer = tf.keras.optimizers.Adam(params['lr'])
    model.compile(loss='mae',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])

    model.fit(Xtr, Ytr, epochs=params['EPOCH'], verbose=0)

    # validation set 예측
    y_pred = model.predict(Xte)
    smape_list = np.zeros((len(y_pred), 1))
    for i in range(0, len(y_pred)):
        smape_list[i] = smape(y_pred[i, :], Yte[i, :])
    avr_smape = np.mean(smape_list)

    # test set 예측
    ypr= model.predict(x_24hrs.reshape(1,-1))
    return ypr, avr_smape, y_pred

def lstm_gen_val(trainAR, testAR, x_24hrs, params = None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    Ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    # default parameter
    if params == None:
        params = {
            'EPOCH': 80,
            'h1': 128,
            'h2': 256,
            'lr': 0.001,
        }
    numData = np.size(trainAR, 0)
    numTr = int(numData * 0.8)
    Xtr = trainAR[0:numTr, :]
    Ytr = testAR[0:numTr, :]

    Xte = trainAR[numTr:numData, :]
    Yte = testAR[numTr:numData, :]

    num_tr = np.size(trainAR, 1)
    num_te = np.size(testAR, 1)

    Xtr = Xtr.reshape(-1, num_tr, 1)
    Xte = Xte.reshape(-1, num_te, 1)
    Ytr = Ytr.reshape(-1, num_tr)
    Yte = Yte.reshape(-1, num_te)
    train_data = tf.data.Dataset.from_tensor_slices((Xtr, Ytr))
    train_data = train_data.shuffle(num_tr, reshuffle_each_iteration=True)
    train_data = train_data.batch(num_tr)
    train_data = train_data.repeat(params['EPOCH'])
    val_data = tf.data.Dataset.from_tensor_slices((Xte, Yte))
    val_data = val_data.batch(num_te)

    def build_model():
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(params['h1'],
                                 return_sequences=False,
                                 input_shape=Xtr.shape[-2:],
                                 activation='relu'),
            layers.Dense(params['h2'], activation='relu'),
            layers.Dense(params['h1'], activation='relu'),
            layers.Dense(num_te)
        ])

        optimizer = tf.keras.optimizers.Adam(0.001)

        model.compile(loss='mae',
                      optimizer=optimizer,
                      metrics=['mae', 'mse'])
        return model

    model = build_model()

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=30,
        verbose=0,
        mode='min',
        baseline=None,
        restore_best_weights=True
    )
    # lstm 모델 fit
    history = model.fit(
        train_data, epochs=params['EPOCH'], verbose=0, validation_data=val_data, callbacks=[es])

    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist.tail()
    # validation set 예측
    y_pred = model.predict(Xte)
    smape_list = np.zeros((len(y_pred), 1))

    for i in range(0, len(y_pred)):
        smape_list[i] = smape(y_pred[i, :], Yte[i, :])
    avr_smape = np.mean(smape_list)
    # test set 예측
    x_24hrs = np.reshape(x_24hrs, (-1,24,1))
    Ypr= model.predict(x_24hrs)
    return Ypr, avr_smape, y_pred

def rf_gen_val(trainAR, testAR, x_24hrs, params = None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    Dnum = trainAR.shape[0]
    lnum = trainAR.shape[1]
    if params == None:
        params = {
            'max_depth': 2,
            'random_state': 0,
            'n_estimators': 100,
            'criterion': 'mae'
        }
    numTr = int(Dnum * 0.8)
    y_pred = np.zeros((Dnum - numTr, 24))
    smape_list = np.zeros([Dnum, 1])
    # cross validation
    for ii in range(0, Dnum):
        trainAR_temp = np.delete(trainAR, ii, axis=0)
        testAR_temp = np.delete(testAR, ii, axis=0)

        regr = RandomForestRegressor(**params)
        regr.fit(trainAR_temp, testAR_temp)

        x_temp = np.zeros([1, lnum])
        for kk in range(0, lnum):
            x_temp[0, kk] = trainAR[ii, kk]

        ypr = regr.predict(x_temp)
        if ii >= numTr:
            y_pred[ii - numTr, :] = ypr
        yre = np.zeros([1, lnum])
        for kk in range(0, lnum):
            yre[0, kk] = testAR[ii, kk]
        smape_list[ii] = smape(np.transpose(ypr), np.transpose(yre))
    # Randomforest 모델 형성 및 예측
    regr = RandomForestRegressor(**params)
    regr.fit(trainAR, testAR)
    x_24hrs = np.reshape(x_24hrs, (-1, lnum))
    ypr = regr.predict(x_24hrs)
    avg_smape = np.mean(smape_list)
    return ypr, avg_smape, y_pred

def svr_gen_val(trainAR, testAR, x_24hrs, params = None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    Dnum = trainAR.shape[0]
    lnum = trainAR.shape[1]
    # default parameter
    if params == None:
        params = {
            'kernel': 'rbf',
        }

    numTr = int(Dnum * 0.8)
    y_pred = np.zeros((Dnum - numTr, 24))
    # cross validation
    smape_list = np.zeros([Dnum, 1])
    for ii in range(0, Dnum):  # cross validation
        trainAR_temp = np.delete(trainAR, ii, axis=0)
        testAR_temp = np.delete(testAR, ii, axis=0)

        regr = MultiOutputRegressor(SVR(**params))
        regr.fit(trainAR_temp, testAR_temp)

        x_temp = np.zeros([1, lnum])
        for kk in range(0, lnum):
            x_temp[0, kk] = trainAR[ii, kk]

        ypr = regr.predict(x_temp)
        if ii >= numTr:
            y_pred[ii - numTr, :] = ypr
        yre = np.zeros([1, lnum])
        for kk in range(0, lnum):
            yre[0, kk] = testAR[ii, kk]
        smape_list[ii] = smape(np.transpose(ypr), np.transpose(yre))

    # validation set 예측
    avg_smape = np.mean(smape_list)
    regr = MultiOutputRegressor(SVR(**params))
    regr.fit(trainAR, testAR)
    x_24hrs = np.reshape(x_24hrs, (-1, lnum))
    # test set 예측
    ypr = regr.predict(x_24hrs)
    return ypr, avg_smape, y_pred

def extra_gen_val(trainAR, testAR, x_24hrs, params = None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    Dnum = trainAR.shape[0]
    lnum = trainAR.shape[1]
    # default parameter
    if params == None:
        params = {
            'n_estimators': 100,
            'min_samples_split': 25,
        }
    numTr = int(Dnum * 0.8)
    y_pred = np.zeros((Dnum - numTr, 24))
    # cross validation
    smape_list = np.zeros([Dnum, 1])
    for ii in range(0, Dnum):  # cross validation
        trainAR_temp = np.delete(trainAR, ii, axis=0)
        testAR_temp = np.delete(testAR, ii, axis=0)

        regr = ExtraTreesRegressor(**params)
        regr.fit(trainAR_temp, testAR_temp)

        x_temp = np.zeros([1, lnum])
        for kk in range(0, lnum):
            x_temp[0, kk] = trainAR[ii, kk]

        ypr = regr.predict(x_temp)
        if ii >= numTr:
            y_pred[ii - numTr, :] = ypr
        yre = np.zeros([1, lnum])
        for kk in range(0, lnum):
            yre[0, kk] = testAR[ii, kk]
        smape_list[ii] = smape(np.transpose(ypr), np.transpose(yre))
    avg_smape = np.mean(smape_list)
    regr = ExtraTreesRegressor(**params)
    regr.fit(trainAR, testAR)
    # test set 예측
    x_24hrs = np.reshape(x_24hrs, (-1, lnum))
    ypr = regr.predict(x_24hrs)
    return ypr, avg_smape, y_pred

def dct_gen_val(trainAR, testAR, x_24hrs, params = None):
    """
    [ Input ]
    trainAR: train 데이터의 예측 전 날 데이터 셋
    testAR:  train 데이터의 예측 날 데이터 셋
    x_24hrs: 예측 전 날의 데이터
    params: 예측 모듈 parameter
    
    [ Output ]
    ypr: 추정하고자하는 날에 대한 예측 프로파일
    avr_smape: validation set의 smape 평균
    y_pred: validation 예측 결과
    """
    Dnum = trainAR.shape[0]
    lnum = trainAR.shape[1]
    if params == None:
        params = {
            'random_state': 0,
        }

    numTr = int(Dnum * 0.8)
    y_pred = np.zeros((Dnum - numTr, 24))
    # cross validation
    smape_list = np.zeros([Dnum, 1])
    for ii in range(0, Dnum):  # cross validation
        trainAR_temp = np.delete(trainAR, ii, axis=0)
        testAR_temp = np.delete(testAR, ii, axis=0)

        regr = DecisionTreeRegressor(**params)
        regr.fit(trainAR_temp, testAR_temp)

        x_temp = np.zeros([1, lnum])
        for kk in range(0, lnum):
            x_temp[0, kk] = trainAR[ii, kk]

        ypr = regr.predict(x_temp)
        if ii >= numTr:
            y_pred[ii - numTr, :] = ypr
        yre = np.zeros([1, lnum])
        for kk in range(0, lnum):
            yre[0, kk] = testAR[ii, kk]
        smape_list[ii] = smape(np.transpose(ypr), np.transpose(yre))
    avg_smape = np.mean(smape_list)
    regr = DecisionTreeRegressor(**params)
    regr.fit(trainAR, testAR)
    # test set 예측
    x_24hrs = np.reshape(x_24hrs, (-1, lnum))
    ypr = regr.predict(x_24hrs)
    return ypr, avg_smape, y_pred

## 3. 모델 학습 및 검증 

* test.csv의 200개 집 데이터에 대해 모델 학습 및 검증을 진행  
1) NaN detection 및 interpolation 수행  
2) AR, Similar day approach, dct, dnn, extra, rf, svr 모델을 학습 및 검증하고, validation set에 대한 smape 결과를 smapes_all에 저장  
3) 저장된 smape 결과를 기반으로 각 집마다 앞선 7개 모델의 앙상블을 통해 하루 24 포인트를 예측    
  


In [8]:
## 결과 저장
test_reals_all = dict() # Validation set에 대한 Grount Truth 
val_preds_all = dict() #  Validation set에 대한 예측 결과
test_preds_all = dict() # Test set에 대한 24포인트 예측 결과
smapes_all = dict() # Validation set에 대한 smape 결과

for key_idx, key in tqdm(enumerate(test.columns)):
    prev_type = 2  # previous day type
    curr_type = 2  # current day type
    trainAR, testAR = tr_data_set(test, key, prev_type, curr_type)

    # 마지막 날 데이터 => nan 처리 과정 => interpolation
    temp_test = test[key].iloc[8759 - 40:]
    temp_test = temp_test.interpolate(method='spline', order=2)
    temp_test = np.array(temp_test.values)
    temp_test = temp_test[len(temp_test) - 24:len(temp_test) + 1]
    subm_24hrs = temp_test
    del temp_test

    fchk = 1  # filter length
    temp_idx = []
    smape_lin = []

    # 선형 예측을 이용해서 NaN이 발견될 시 제거
    for chk_bad in range(0, len(trainAR[:, 0])):
        prev_smape = 200  # 기준 smape
        nan_chk = 0

        trainAR_temp = np.zeros([1, 24])
        testAR_temp = np.zeros([1, 24])

        for ii in range(0, 24):
            trainAR_temp[0, ii] = trainAR[chk_bad, ii]
            testAR_temp[0, ii] = testAR[chk_bad, ii]

        # 선형 예측
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR_temp, testAR_temp, fchk, subm_24hrs)

        # NaN 처리
        if np.isnan(lin_sampe):
            nan_chk = 1
        if np.isnan(np.sum(trainAR_temp)):
            nan_chk = 1
        if np.isnan(np.sum(testAR_temp)):
            nan_chk = 1
        if nan_chk == 1:
            temp_idx.append(chk_bad)

    # NaN 값이 나타난 data set은 제거
    trainAR = np.delete(trainAR, temp_idx, axis=0)
    testAR = np.delete(testAR, temp_idx, axis=0)

    del_smape = np.zeros([1, len(trainAR[:, 1])])
    prev_smape = 200
    fchk = 0

    # 선형 예측 시, filter length 선정
    for chk in range(3, 24):
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR, testAR, chk, subm_24hrs)
        if prev_smape > lin_sampe:
            fchk = chk
            prev_smape = lin_sampe

    # 최적의 smape를 탐색
    for chk_lin in range(0, len(trainAR[:, 1])):
        trainAR_temp = np.delete(trainAR, chk_lin, axis=0)
        testAR_temp = np.delete(testAR, chk_lin, axis=0)
        lin_sampe, fcst_temp, pred_hyb = linear_prediction(trainAR_temp, testAR_temp, fchk, subm_24hrs)
        del_smape[0, chk_lin] = lin_sampe

    # smape의 성능을 감소시키는 행 제거
    trainAR = np.delete(trainAR, np.argmin(del_smape), axis=0)
    testAR = np.delete(testAR, np.argmin(del_smape), axis=0)
    del_smape = np.delete(del_smape, np.argmin(del_smape), axis=1)

    del nan_chk, lin_sampe, fcst_temp, pred_hyb, prev_smape, temp_idx

    ## 결과 저장
    smape_results = dict()
    val_preds = dict()
    test_preds = dict()

    ## baseline results
    # AR
    fcst, smape_val, y_pred = AR_val(trainAR, testAR, fchk, subm_24hrs)
    smape_results['AR'], val_preds['AR'],test_preds['AR'] = smape_val, y_pred, fcst

    # Similar day
    temp_24hrs = np.zeros([1, 24])
    for qq in range(0, 24):
        temp_24hrs[0, qq] = subm_24hrs[qq]
    prev_smape = 200
    fsim = 0  # N개의 날
    for sim_len in range(2, 5):
        sim_fcst, sim_smape, y_pred = similar_approach_val(trainAR, testAR, sim_len, temp_24hrs)
        if prev_smape > sim_smape:
            fsim = sim_len
            prev_smape = sim_smape
    fcst, smape_val, y_pred = similar_approach_val(trainAR, testAR, fsim, temp_24hrs)
    smape_results['Sim'], val_preds['Sim'], test_preds['Sim'] = smape_val, y_pred, fcst

    ## Tuning된 parameter를 불러옴
    dct_param = dct_param_list.loc[key,:]
    dnn_param = dnn_param_list.loc[key,:]
    extra_param = extra_param_list.loc[key,:]
    rf_param = rf_param_list.loc[key,:]
    svr_param = svr_param_list.loc[key,:]
    param_list = [dct_param, dnn_param, extra_param, rf_param, svr_param]
    method_list = ['dct','dnn','extra','rf','svr']

    # Decision tree, DNN, LSTM, Extra tree, Ranfom Forest, SVR 모델 생성 및 예측
    Dnum = trainAR.shape[0]
    numTr = int(Dnum * 0.8)
    trues = testAR[numTr:,:]
    methods = []
    for method_idx in range(5):
        params = param_list[method_idx]
        method = method_list[method_idx]
        params = params.to_dict()
        if method == 'dnn': # dnn일 경우 dnn과 lstm 중 성능이 좋은 모델을 선택
            fcst_list, smape_list, y_pred_list = [], [], []
            for i in range(11):
                params['EPOCH'] = int(params['EPOCH'])
                fcst, smape_val, y_pred = dnn_gen_val(trainAR, testAR, subm_24hrs, params=params)
                # lstm
                if i == 10:
                    fcst, smape_val, y_pred = lstm_gen_val(trainAR, testAR, subm_24hrs, params = {'EPOCH': 80, 'h1': 432, 'h2': 168, 'lr': 0.0004})
                fcst_list.append(fcst)
                smape_list.append(smape_val)
                y_pred_list.append(y_pred)
                tf.keras.backend.clear_session()
            best_idx = np.argmin(smape_list, axis=0)
            smape_val = smape_list[best_idx]
            fcst = fcst_list[best_idx]
            y_pred = y_pred_list[best_idx]
        elif method == 'svr':
            fcst, smape_val, y_pred = svr_gen_val(trainAR, testAR, subm_24hrs, params=params)
        elif method == 'rf':
            fcst, smape_val, y_pred = rf_gen_val(trainAR, testAR, subm_24hrs, params=params)
        elif method == 'dct':
            fcst, smape_val, y_pred = dct_gen_val(trainAR, testAR, subm_24hrs, params=params)
        elif method == 'extra':
            fcst, smape_val, y_pred = extra_gen_val(trainAR, testAR, subm_24hrs, params=params)
        # save results
        key_name = method
        smape_results[key_name], val_preds[key_name], test_preds[key_name] = smape_val, y_pred, fcst

    test_reals_all[key] = testAR[numTr:,:]
    val_preds_all[key] = val_preds
    test_preds_all[key] = test_preds
    smapes_all[key] = smape_results
    print(f'smape result is {smape_results}')

0it [00:00, ?it/s]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1it [02:22, 142.50s/it]

smape result is {'AR': 17.26631835967926, 'Sim': 18.834771710873188, 'dct': 21.59511230321437, 'dnn': 16.05974799749037, 'extra': 21.595112303214343, 'rf': 21.903082259360026, 'svr': 20.910853546261006}


2it [04:26, 136.91s/it]

...

smape result is {'AR': 54.52553675838886, 'Sim': 36.712838404875136, 'dct': 35.99636606883503, 'dnn': 44.73184834041164, 'extra': 35.45560012913838, 'rf': 37.817491517973465, 'svr': 37.71101787941636}


199it [11:47:17, 338.65s/it]

smape result is {'AR': 34.71982443174648, 'Sim': 34.27271904162747, 'dct': 17.97260694011017, 'dnn': 3.828386487829634, 'extra': 15.621128385364516, 'rf': 16.753590260925463, 'svr': 23.512829622756247}


200it [11:50:34, 213.17s/it]

smape result is {'AR': 30.441564704332677, 'Sim': 25.736186860709218, 'dct': 25.85625350658269, 'dnn': 25.858795615443974, 'extra': 24.68463741674599, 'rf': 26.511780100758436, 'svr': 25.082871652444503}


In [9]:
#%% 24시간의 결과 앙상블
for key_idx, key in enumerate(test.columns):
    # load results
    test_preds = test_preds_all[key]
    true = test_reals_all[key]
    val_preds = val_preds_all[key]
    num_methods = len(val_preds)

    ## smape loss 기준으로 weight 결정
    methods = np.array(list(smapes_all[key].keys()))
    loss = np.array(list(smapes_all[key].values()))
    drop_idx = loss > np.min(loss) * 1.1
    weight = np.ones((num_methods))
    weight[drop_idx] = 0
    weight[loss < np.min(loss) * 1.05] = 3
    weight[np.argmin(loss)] = 5
    weight /= weight.sum()
    print(weight)

    ## test prediction based on weight
    ensemble_pred = np.zeros((1, 24))
    for j in range(num_methods):
        ensemble_pred += weight[j] * test_preds[methods[j]]

    ## submission에 저장
    submission.loc[key, submission.columns[0]:submission.columns[23]] = np.ravel(ensemble_pred)


[0.16666667 0.         0.         0.83333333 0.         0.
 0.        ]
[0.         0.55555556 0.11111111 0.         0.11111111 0.11111111
 0.11111111]
[0. 0. 0. 1. 0. 0. 0.]
[0.  0.3 0.  0.5 0.1 0.  0.1]
[0.         0.55555556 0.11111111 0.         0.11111111 0.11111111
 0.11111111]
[0.         0.         0.         0.25       0.25       0.08333333
 0.41666667]
[0.         0.16666667 0.         0.83333333 0.         0.
 0.        ]
[0.         0.21428571 0.07142857 0.35714286 0.07142857 0.07142857
 0.21428571]
[0.         0.         0.         0.14285714 0.71428571 0.
 0.14285714]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.         0.         0.27272727 0.27272727 0.
 0.45454545]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.         0.11111111 0.         0.33333333 0.
 0.55555556]
[0.         0.16666667 0.         0.83333333 0.         0.
 0.        ]
[0.         0.16666667 0.         0.83333333 0.         0.
 0.        ]
[0.  0.3 0.  0.5 0.1 0.  0.1]
[0.         0.16666667 0

 0.11111111]
[0.    0.125 0.    0.625 0.125 0.    0.125]
[0.  0.  0.1 0.1 0.3 0.  0.5]
[0.         0.27272727 0.         0.45454545 0.09090909 0.09090909
 0.09090909]
[0.     0.0625 0.1875 0.1875 0.1875 0.3125 0.0625]
[0.         0.07692308 0.         0.38461538 0.23076923 0.07692308
 0.23076923]
[0.     0.1875 0.1875 0.0625 0.1875 0.0625 0.3125]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.11111111 0.11111111 0.11111111 0.55555556 0.
 0.11111111]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.38461538 0.07692308 0.23076923 0.23076923 0.
 0.07692308]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.         0.         0.71428571 0.14285714 0.
 0.14285714]
[0. 0. 0. 1. 0. 0. 0.]
[0.    0.375 0.    0.625 0.    0.    0.   ]
[0.11111111 0.         0.         0.         0.33333333 0.
 0.55555556]
[0. 0. 0. 1. 0. 0. 0.]
[0.         0.29411765 0.17647059 0.         0.17647059 0.17647059

* 일, 월 예측은 Similar day approach를 통해 수행

In [10]:
for key_idx, key in enumerate(test.columns):
    ### Day prediction
    fcst_d = np.zeros([1, 10])

    trainAR_Day = tr_day_set(test, key)

    # Similar day approach
    day_idx = np.zeros([1, 10])
    for ii in range(0, 10):
        mod_idx = -1
        # 요일 idx
        temp_idx = (ii + mod_idx) % 7
        day_idx[0, ii] = temp_idx

    for ii in range(0, 10):
        flen = np.random.randint(3) + 2
        temp_day = np.zeros([1, flen])
        for jj in range(0, flen):
            temp_day[0, jj] = trainAR_Day[jj, int(round(day_idx[0, ii]))]
        # 평균값으로 예측
        fcst_d[0, ii] = np.mean(temp_day)

    # submission에 저장
    for i in range(10):
        submission.loc[key,'X2018_7_' + str(i + 1) + '_d'] = fcst_d[0][i]

    del mod_idx, temp_idx

    ### Month prediction
    mon_test = np.zeros([1, 300])

    # Similar day approach
    day_idx = np.zeros([1, 300])
    for ii in range(0, 300):
        mod_idx = -1
        # 요일 idx 생성(월~일: 0~6)
        temp_idx = (ii + mod_idx) % 7
        day_idx[0, ii] = temp_idx

    # 휴일의 경우, 일요일과 같은 데이터로 가정함.
    day_idx[0, 31 + 15 - 1] = 6  # 광복절
    day_idx[0, 31 + 31 + 24 - 1] = 6  # 추석
    day_idx[0, 31 + 31 + 25 - 1] = 6  # 추석
    day_idx[0, 31 + 31 + 26 - 1] = 6  # 대체휴일
    day_idx[0, 31 + 31 + 30 + 3 - 1] = 6  # 개천절
    day_idx[0, 31 + 31 + 30 + 9 - 1] = 6  # 한글날
    day_idx[0, 31 + 31 + 30 + 31 + 30 + 25 - 1] = 6  # 성탄절

    # 1~4주 전의 같은 요일 데이터를 랜덤하게 불러와 평균을 취함
    for ii in range(0, 300):
        flen = np.random.randint(3) + 1
        temp_day = np.zeros([1, flen])
        for jj in range(0, flen):
            temp_day[0, jj] = trainAR_Day[jj, int(round(day_idx[0, ii]))]
        mon_test[0, ii] = np.mean(temp_day)

    # 결과 합
    pred_7m = np.sum(mon_test[0, 0:31])
    pred_8m = np.sum(mon_test[0, 31:62])
    pred_9m = np.sum(mon_test[0, 62:92])
    pred_10m = np.sum(mon_test[0, 92:123])
    pred_11m = np.sum(mon_test[0, 123:153])

    submission.loc[key,'X2018_7_m'] = pred_7m
    submission.loc[key,'X2018_8_m'] = pred_8m
    submission.loc[key,'X2018_9_m'] = pred_9m
    submission.loc[key,'X2018_10_m'] = pred_10m
    submission.loc[key,'X2018_11_m'] = pred_11m

submission.to_csv('submission.csv')

## 4. 결과
* 다양한 모델의 앙상블을 통해 향상된 예측 결과를 얻을 수 있음  
* public score: 27.41, private score: 26.98